In [1]:
import pandas as pd
import numpy as np
name = "billing_and_accruals_for_script_2024-10-01_06-15-57"

In [2]:
dtype_dict = {
    'КПП': str,
}

In [3]:
dataset = pd.read_excel('C:/Users/Boris/Downloads/' + name + '.xlsx', dtype=dtype_dict)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9863 entries, 0 to 9862
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Название компании           9783 non-null   object        
 1   Дата                        9863 non-null   datetime64[ns]
 2   Личный кабинет              9863 non-null   int64         
 3   ЛС                          9696 non-null   object        
 4   ИНН                         8811 non-null   object        
 5   КПП                         5642 non-null   object        
 6   Номер договора              9696 non-null   object        
 7   Идентификатор номенклатуры  9696 non-null   float64       
 8   Название номенклатуры       9696 non-null   object        
 9   Стоимость                   9863 non-null   float64       
 10  Сумма                       9863 non-null   float64       
 11  Начисления partner          4379 non-null   float64     

In [5]:
dataset['КПП'] = dataset['КПП'].astype(str)

In [6]:
dataset.head(2)

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner
0,"ООО ""Видеоглаз Вэст""",2022-03-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,NaN
1,"ООО ""Видеоглаз Вэст""",2022-04-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,NaN


In [7]:
dataset['Начисления partner'] = dataset['Начисления partner'].fillna(0)

In [8]:
dataset['Сумма-test'] = dataset['Сумма']

In [9]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test
0,"ООО ""Видеоглаз Вэст""",2022-03-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,0.0,290.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,0.0,230.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9858,"ТСН ""Зеленая, 25""",2024-09-30,153682,153682,3702271261,370201001,153682,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,68.0,0.0,68.0
9859,"ТСН ""Зеленая, 25""",2024-10-31,153682,153682,3702271261,370201001,153682,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,4.0,0.0,4.0
9860,ИП Усов В.Л.,2024-09-30,153694,Подписка ПРО ЛК 153694,331101193492,nan,Подписка ПРО ЛК 153694,40.0,Подписка ПРО,2.7,29.7,0.0,29.7
9861,ИП Усов В.Л.,2024-10-31,153694,Подписка ПРО ЛК 153694,331101193492,nan,Подписка ПРО ЛК 153694,40.0,Подписка ПРО,2.7,2.7,0.0,2.7


In [10]:
for i in range(0,dataset.shape[0]):
    if i == 0:
        date = dataset.loc[0,'Дата']
        lk = dataset.loc[0,'Личный кабинет'] 
        dataset.loc[0,'ballance'] = 0
        dataset.loc[0,'ballance'] = dataset.loc[0,'Сумма'] - dataset.loc[0, 'Начисления partner']
        if dataset.loc[0,'ballance'] < 0:
            dataset.loc[0,'Сумма']  = 0
        if dataset.loc[0,'ballance'] >= 0:
                dataset.loc[0,'Сумма'] = dataset.loc[0,'ballance'] 
                dataset.loc[0,'ballance'] = 0
            
    else:
    
        if date == dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if date != dataset.loc[i,'Дата'] and lk == dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] - dataset.loc[i, 'Начисления partner'] + dataset.loc[i-1,'ballance']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0
        
        if lk != dataset.loc[i,'Личный кабинет']:  
            date = dataset.loc[i,'Дата']
            lk = dataset.loc[i,'Личный кабинет']
            
            dataset.loc[i,'ballance'] = dataset.loc[i,'Сумма'] \
                                        - dataset.loc[i, 'Начисления partner']
            if dataset.loc[i,'ballance'] < 0:
                dataset.loc[i,'Сумма']  = 0
                
            if dataset.loc[i,'ballance'] >= 0:
                dataset.loc[i,'Сумма'] = dataset.loc[i,'ballance']
                dataset.loc[i,'ballance'] = 0



In [11]:
dataset

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Начисления partner,Сумма-test,ballance
0,"ООО ""Видеоглаз Вэст""",2022-03-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,0.0,290.0,0.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0,0.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,0.0,300.0,0.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,0.0,230.0,0.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9858,"ТСН ""Зеленая, 25""",2024-09-30,153682,153682,3702271261,370201001,153682,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,68.0,0.0,68.0,0.0
9859,"ТСН ""Зеленая, 25""",2024-10-31,153682,153682,3702271261,370201001,153682,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,4.0,0.0,4.0,0.0
9860,ИП Усов В.Л.,2024-09-30,153694,Подписка ПРО ЛК 153694,331101193492,nan,Подписка ПРО ЛК 153694,40.0,Подписка ПРО,2.7,29.7,0.0,29.7,0.0
9861,ИП Усов В.Л.,2024-10-31,153694,Подписка ПРО ЛК 153694,331101193492,nan,Подписка ПРО ЛК 153694,40.0,Подписка ПРО,2.7,2.7,0.0,2.7,0.0


In [12]:
dataset.to_excel('C:/Users/Boris/Downloads/'+ name + '_test.xlsx', index=False)

In [13]:
dataset_ready = dataset.drop(['Начисления partner','Сумма-test','ballance'], axis= 1)

In [14]:
dataset_ready['Количество'] = round(dataset_ready['Сумма']/dataset_ready['Стоимость'])
dataset_ready['Количество'] = dataset_ready['Количество'].fillna(0) 

In [15]:
dataset_ready

,Название компании,Дата,Личный кабинет,ЛС,ИНН,КПП,Номер договора,Идентификатор номенклатуры,Название номенклатуры,Стоимость,Сумма,Количество
0,"ООО ""Видеоглаз Вэст""",2022-03-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,290.0,29.0
1,"ООО ""Видеоглаз Вэст""",2022-04-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,30.0
2,"ООО ""Видеоглаз Вэст""",2022-05-31,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,300.0,30.0
3,"ООО ""Видеоглаз Вэст""",2022-06-30,120066,NaN,507406410220,nan,NaN,NaN,NaN,10.0,230.0,23.0
4,"ООО ""ИНКО-ТЕЛЕКОМ""",2021-06-30,120068,120068,NaN,nan,120068,80.0,Предоставление доступа к функционалу Энтерпрайз,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9858,"ТСН ""Зеленая, 25""",2024-09-30,153682,153682,3702271261,370201001,153682,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,68.0,17.0
9859,"ТСН ""Зеленая, 25""",2024-10-31,153682,153682,3702271261,370201001,153682,10.0,Архивация видеопотока с заданной глубиной хран...,4.0,4.0,1.0
9860,ИП Усов В.Л.,2024-09-30,153694,Подписка ПРО ЛК 153694,331101193492,nan,Подписка ПРО ЛК 153694,40.0,Подписка ПРО,2.7,29.7,11.0
9861,ИП Усов В.Л.,2024-10-31,153694,Подписка ПРО ЛК 153694,331101193492,nan,Подписка ПРО ЛК 153694,40.0,Подписка ПРО,2.7,2.7,1.0


In [16]:
dataset_ready.to_excel('C:/Users/Boris/Downloads/'+ name + '_ready.xlsx', index=False)